# Daily Version Tracking Pipeline - Production Ready

## Purpose

This notebook implements the **PRODUCTION PIPELINE** for daily version tracking:

1. ✅ Detects new/updated content from `content_repo.db`
2. ✅ Compares checksums to identify changes
3. ✅ Creates version history records automatically
4. ✅ Builds version chains (links previous versions)
5. ✅ Deprecates affected FAQs
6. ✅ Runs incrementally (only processes changes since last run)

## Difference from `version_tracking_implementation.ipynb`

| Feature | version_tracking_implementation.ipynb | This Notebook |
|---------|--------------------------------------|---------------|
| Purpose | Educational/Query Examples | Production Pipeline |
| Data Source | Pre-populated sample data | Database (content_repo.db) |
| Version Detection | ❌ No | ✅ Yes |
| Change Detection | ❌ No | ✅ Yes |
| Auto-Create Versions | ❌ No | ✅ Yes |
| Incremental Updates | ❌ No | ✅ Yes |
| Table Creation | Creates every run | Once on first run |
| FAQ Deprecation | ❌ No | ✅ Yes |

## Prerequisites

⚠️ **IMPORTANT**: You must run `setup_databases.ipynb` FIRST to create the required databases.

## Daily Workflow

```
Day 1: content_repo.db updated → Run pipeline → Detect changes → Create version records
Day 2: content_repo.db updated → Run pipeline → Detect changes → Create version records
...
```

## Step 1: Setup and Configuration

**Key Change**: We load content directly from `content_repo.db` (no CSV needed) and calculate checksums from actual markdown files.

In [26]:
import pandas as pd
import sqlite3
import json
import hashlib
from pathlib import Path
from datetime import datetime, timedelta
import difflib

# Configuration
DOCS_PATH = Path('../Docs')
DATABASES_DIR = Path('../databases')
SCHEMA_SQL = DOCS_PATH / 'schema_content_version_tracking.sql'

# Database paths (all created by setup_databases.ipynb)
CONTENT_REPO_DB = DATABASES_DIR / 'content_repo.db'
FAQ_QUESTION_DB = DATABASES_DIR / 'faq_question.db'
FAQ_ANSWER_DB = DATABASES_DIR / 'faq_answer.db'
VERSION_TRACKING_DB = DATABASES_DIR / 'version_tracking.db'

# Pipeline parameters
LAST_RUN_DATE_FILE = DATABASES_DIR / 'last_run_date.txt'

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Verify databases exist
print("🔍 Checking required databases...")
missing_dbs = []
for db_path in [CONTENT_REPO_DB, FAQ_QUESTION_DB, FAQ_ANSWER_DB, VERSION_TRACKING_DB]:
    if not db_path.exists():
        missing_dbs.append(db_path.name)
        print(f"  ❌ {db_path.name} - NOT FOUND")
    else:
        print(f"  ✅ {db_path.name} - Found")

if missing_dbs:
    print(f"\n❌ ERROR: Missing databases: {', '.join(missing_dbs)}")
    print("   Please run setup_databases.ipynb first to create all required databases.")
    raise FileNotFoundError(f"Required databases not found: {missing_dbs}")

def calculate_file_checksum(file_path):
    """
    Calculate SHA-256 checksum from actual file content.
    This ensures we detect REAL changes, not just metadata updates.
    """
    try:
        with open(file_path, 'rb') as f:
            content = f.read()
            return hashlib.sha256(content).hexdigest()
    except FileNotFoundError:
        print(f"⚠️ Warning: File not found: {file_path}")
        return None
    except Exception as e:
        print(f"⚠️ Error reading file {file_path}: {e}")
        return None

print("\n✅ Configuration loaded")
print(f"📁 Docs Path: {DOCS_PATH}")
print(f"🗄️ Databases Directory: {DATABASES_DIR}")
print(f"   - Content Repo DB: {CONTENT_REPO_DB.name}")
print(f"   - FAQ Question DB: {FAQ_QUESTION_DB.name}")
print(f"   - FAQ Answer DB: {FAQ_ANSWER_DB.name}")
print(f"   - Version Tracking DB: {VERSION_TRACKING_DB.name}")
print(f"🔐 Checksum calculation: From actual markdown files (not database checksums)")

🔍 Checking required databases...
  ✅ content_repo.db - Found
  ✅ faq_question.db - Found
  ✅ faq_answer.db - Found
  ✅ version_tracking.db - Found

✅ Configuration loaded
📁 Docs Path: ..\Docs
🗄️ Databases Directory: ..\databases
   - Content Repo DB: content_repo.db
   - FAQ Question DB: faq_question.db
   - FAQ Answer DB: faq_answer.db
   - Version Tracking DB: version_tracking.db
🔐 Checksum calculation: From actual markdown files (not database checksums)


## Step 2: Connect to Version Tracking Database

**IMPORTANT**: This notebook does NOT create databases or schema. It only uses existing databases created by `setup_databases.ipynb`.

The database and schema should already exist from `setup_databases.ipynb`.

In [27]:
# Connect to version_tracking.db (must already exist from setup_databases.ipynb)
if not VERSION_TRACKING_DB.exists():
    raise FileNotFoundError(
        f"❌ Database not found: {VERSION_TRACKING_DB}\n"
        f"Please run setup_databases.ipynb first to create all databases."
    )

# Verify schema exists (read-only check) - use temporary connection
with sqlite3.connect(VERSION_TRACKING_DB) as conn_verify:
    cursor = conn_verify.cursor()
    cursor.execute("""
        SELECT name FROM sqlite_master
        WHERE type='table' AND name='content_version_history'
    """)
    
    if cursor.fetchone() is None:
        raise RuntimeError(
            f"❌ Schema not found in {VERSION_TRACKING_DB}\n"
            f"The database exists but tables are missing.\n"
            f"Please run setup_databases.ipynb to apply the schema."
        )
    
    # Verify all required tables exist (read-only check)
    required_tables = ['content_version_history', 'faq_content_link', 'faq_validation_history', 'content_change_impact']
    tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn_verify)
    existing_tables = tables['name'].tolist()
    
    missing_tables = [t for t in required_tables if t not in existing_tables]
    if missing_tables:
        raise RuntimeError(
            f"❌ Missing required tables: {', '.join(missing_tables)}\n"
            f"Please run setup_databases.ipynb to create all tables."
        )
    
    print(f"✅ Connected to version_tracking.db")
    print(f"📊 All required tables exist: {', '.join(required_tables)}")
# Auto-closes here

✅ Connected to version_tracking.db
📊 All required tables exist: content_version_history, faq_content_link, faq_validation_history, content_change_impact


## Step 3: Load Last Run Date

Determine which changes to process based on last pipeline run.

In [28]:
def get_last_run_date(file_path):
    """
    Get the last pipeline run date.
    If file doesn't exist, use a date far in the past (process all).
    """
    if file_path.exists():
        with open(file_path, 'r') as f:
            last_run = f.read().strip()
        print(f"📅 Last run: {last_run}")
        return last_run
    else:
        # First run - process everything
        default_date = "2020-01-01T00:00:00.000Z"
        print(f"📅 First run detected. Processing all content since {default_date}")
        return default_date

def save_run_date(file_path, run_date):
    """
    Save the current run date for next execution.
    """
    file_path.parent.mkdir(parents=True, exist_ok=True)
    with open(file_path, 'w') as f:
        f.write(run_date)
    print(f"💾 Saved run date: {run_date}")

# Get last run date
LAST_RUN_DATE = get_last_run_date(LAST_RUN_DATE_FILE)
CURRENT_RUN_DATE = datetime.now().isoformat() + 'Z'

print(f"🕐 Current run: {CURRENT_RUN_DATE}")
print(f"\n🔍 Will process content modified between {LAST_RUN_DATE} and {CURRENT_RUN_DATE}")

📅 First run detected. Processing all content since 2020-01-01T00:00:00.000Z
🕐 Current run: 2025-10-05T00:13:44.586723Z

🔍 Will process content modified between 2020-01-01T00:00:00.000Z and 2025-10-05T00:13:44.586723Z


## Step 4: Load Data from Databases

Load content repository and existing version history from databases.

In [29]:
# Load content repository from content_repo.db (not CSV!)
with sqlite3.connect(CONTENT_REPO_DB) as conn_content:
    content_repo = pd.read_sql("SELECT * FROM content_repo", conn_content)
print(f"📊 Loaded content_repo from database: {len(content_repo)} records")

# Calculate checksums from actual files (not from database checksum column)
print(f"\n🔐 Calculating checksums from markdown files...")
checksums_calculated = 0
checksums_failed = 0

for idx, row in content_repo.iterrows():
    markdown_path = row['extracted_markdown_file_path']
    if pd.notna(markdown_path):
        checksum = calculate_file_checksum(markdown_path)
        if checksum:
            content_repo.at[idx, 'calculated_checksum'] = checksum
            checksums_calculated += 1
        else:
            content_repo.at[idx, 'calculated_checksum'] = None
            checksums_failed += 1
    else:
        content_repo.at[idx, 'calculated_checksum'] = None
        checksums_failed += 1

print(f"  ✅ Calculated: {checksums_calculated}")
print(f"  ⚠️ Failed/Missing: {checksums_failed}")

# Load existing version history from version_tracking database
try:
    with sqlite3.connect(VERSION_TRACKING_DB) as conn_version:
        content_version_history = pd.read_sql("SELECT * FROM content_version_history", conn_version)
    print(f"\n📜 Existing version history: {len(content_version_history)} records")
except:
    content_version_history = pd.DataFrame()
    print("\n📜 No existing version history found (first run)")

# Load FAQ link data from version_tracking database
try:
    with sqlite3.connect(VERSION_TRACKING_DB) as conn_faq_link:
        faq_content_link = pd.read_sql("SELECT * FROM faq_content_link", conn_faq_link)
    print(f"🔗 Existing FAQ links: {len(faq_content_link)} records")
except:
    faq_content_link = pd.DataFrame()
    print("🔗 No existing FAQ links found")

# Load FAQ questions from separate database
print(f"\n📊 Loading FAQ data from separate databases...")
try:
    with sqlite3.connect(FAQ_QUESTION_DB) as conn_question:
        faq_questions = pd.read_sql("SELECT * FROM faq_questions", conn_question)
    print(f"  ✅ FAQ questions: {len(faq_questions)} records")
except Exception as e:
    faq_questions = pd.DataFrame()
    print(f"  ⚠️ Could not load FAQ questions: {e}")

# Load FAQ answers from separate database
try:
    with sqlite3.connect(FAQ_ANSWER_DB) as conn_answer:
        faq_answers = pd.read_sql("SELECT * FROM faq_answers", conn_answer)
    print(f"  ✅ FAQ answers: {len(faq_answers)} records")
except Exception as e:
    faq_answers = pd.DataFrame()
    print(f"  ⚠️ Could not load FAQ answers: {e}")

print("\n📄 Sample content_repo records (with calculated checksum):")
content_repo[['ud_source_file_id', 'raw_file_nme', 'raw_file_page_nbr', 'calculated_checksum', 'last_modified_dt']].head(3)

📊 Loaded content_repo from database: 6 records

🔐 Calculating checksums from markdown files...
  ✅ Calculated: 6
  ⚠️ Failed/Missing: 0

📜 Existing version history: 3 records
🔗 Existing FAQ links: 0 records

📊 Loading FAQ data from separate databases...
  ✅ FAQ questions: 8 records
  ✅ FAQ answers: 5 records

📄 Sample content_repo records (with calculated checksum):


ud_source_file_id               raw_file_nme  raw_file_page_nbr  \
0               1001  Employee_Leave_Policy.pdf                  1   
1               1002  Employee_Leave_Policy.pdf                  2   
2               1003  Employee_Leave_Policy.pdf                  3   

                                 calculated_checksum          last_modified_dt  
0  e51d37e036b08f20b282259753e3435189f8f3e8d47645...  2025-01-01T10:00:00.000Z  
1  90d6029155f6338635e1ab20ba1ae53f428e97ef35b4f7...  2025-01-01T10:00:00.000Z  
2  9b21c11acaf8985e7ba994f23f134ecee5c5f7777ea63f...  2025-01-01T10:00:00.000Z

## Step 5: Filter Changed Content

Identify content that has been modified since last run.

In [30]:
# Filter content modified since last run
modified_content = content_repo[
    content_repo['last_modified_dt'] > LAST_RUN_DATE
]

print(f"🔍 Found {len(modified_content)} modified content records since {LAST_RUN_DATE}")

if len(modified_content) > 0:
    print("\n📋 Modified content summary:")
    summary = modified_content.groupby('raw_file_nme').size().reset_index(name='pages_modified')
    for _, row in summary.iterrows():
        print(f"  - {row['raw_file_nme']}: {row['pages_modified']} pages")
    
    print("\n📄 Sample modified records:")
    print(modified_content[['ud_source_file_id', 'raw_file_nme', 'raw_file_page_nbr', 'last_modified_dt']].head())
else:
    print("\n✅ No changes detected. Pipeline can exit early.")

🔍 Found 6 modified content records since 2020-01-01T00:00:00.000Z

📋 Modified content summary:
  - Employee_Leave_Policy.pdf: 6 pages

📄 Sample modified records:
   ud_source_file_id               raw_file_nme  raw_file_page_nbr  \
0               1001  Employee_Leave_Policy.pdf                  1   
1               1002  Employee_Leave_Policy.pdf                  2   
2               1003  Employee_Leave_Policy.pdf                  3   
3               1002  Employee_Leave_Policy.pdf                  2   
4               1004  Employee_Leave_Policy.pdf                  3   

           last_modified_dt  
0  2025-01-01T10:00:00.000Z  
1  2025-01-01T10:00:00.000Z  
2  2025-01-01T10:00:00.000Z  
3  2025-01-02T10:00:00.000Z  
4  2025-01-03T10:00:00.000Z  


In [31]:
modified_content

ud_source_file_id domain            service  orgoid associateoid  \
0               1001     HR  Policy Management  ORG001     ASSOC001   
1               1002     HR  Policy Management  ORG001     ASSOC001   
2               1003     HR  Policy Management  ORG001     ASSOC001   
3               1002     HR  Policy Management  ORG001     ASSOC001   
4               1004     HR  Policy Management  ORG001     ASSOC001   
5               1003     HR  Policy Management  ORG001     ASSOC001   

                raw_file_nme raw_file_type  raw_file_version_nbr  \
0  Employee_Leave_Policy.pdf          .pdf                     1   
1  Employee_Leave_Policy.pdf          .pdf                     1   
2  Employee_Leave_Policy.pdf          .pdf                     1   
3  Employee_Leave_Policy.pdf          .pdf                     2   
4  Employee_Leave_Policy.pdf          .pdf                     1   
5  Employee_Leave_Policy.pdf          .pdf                     2   

  source_url_txt parent_location_txt                    raw_file_path  \
0           None                None  /test/Employee_Leave_Policy.pdf   
1           None                None  /test/Employee_Leave_Policy.pdf   
2           None                None  /test/Employee_Leave_Policy.pdf   
3           None                None  /test/Employee_Leave_Policy.pdf   
4           None                None  /test/Employee_Leave_Policy.pdf   
5           None                None  /test/Employee_Leave_Policy.pdf   

                        extracted_markdown_file_path  \
0  C:\Users\praka\projects\FAQ\test_data_version_...   
1  C:\Users\praka\projects\FAQ\test_data_version_...   
2  C:\Users\praka\projects\FAQ\test_data_version_...   
3  C:\Users\praka\projects\FAQ\test_data_version_...   
4  C:\Users\praka\projects\FAQ\test_data_version_...   
5  C:\Users\praka\projects\FAQ\test_data_version_...   

  extracted_layout_file_path  raw_file_page_nbr              title_nme  \
0              layout_p1.txt                  1  Employee Leave Policy   
1              layout_p2.txt                  2  Employee Leave Policy   
2              layout_p3.txt                  3  Employee Leave Policy   
3           layout_p2_v2.txt                  2  Employee Leave Policy   
4          layout_p3_new.txt                  3  Employee Leave Policy   
5              layout_p3.txt                  4  Employee Leave Policy   

  breadcrumb_txt content_tags_txt  version_nbr  \
0           None             None            1   
1           None             None            1   
2           None             None            1   
3           None             None            2   
4           None             None            2   
5           None             None            1   

                                    content_checksum file_status  \
0  620c75e45bb1c3a1225a411c9821805c866c60fe8e6a61...      Active   
1  b04ed99ce1bf45e408ed7a519c4d4d65554fcccaa7e3fb...      Active   
2  1e4bd8f57457cee3cb79d7b43f8e8cd4853e7360146738...      Active   
3  ce9d64d784d0d2618919b6d3a49601785095b21f968c06...      Active   
4  4aefd3fb027461ca50f27186787cc28e0d1a90c91ed476...      Active   
5  d92caac90c92e96ed90ae645a8a12d715a47bc33dcdf69...      Active   

                 created_dt          last_modified_dt  \
0  2025-01-01T10:00:00.000Z  2025-01-01T10:00:00.000Z   
1  2025-01-01T10:00:00.000Z  2025-01-01T10:00:00.000Z   
2  2025-01-01T10:00:00.000Z  2025-01-01T10:00:00.000Z   
3  2025-01-01T10:00:00.000Z  2025-01-02T10:00:00.000Z   
4  2025-01-03T10:00:00.000Z  2025-01-03T10:00:00.000Z   
5  2025-01-01T10:00:00.000Z  2025-01-03T10:00:00.000Z   

                                 calculated_checksum  
0  e51d37e036b08f20b282259753e3435189f8f3e8d47645...  
1  90d6029155f6338635e1ab20ba1ae53f428e97ef35b4f7...  
2  9b21c11acaf8985e7ba994f23f134ecee5c5f7777ea63f...  
3  02d546c7c11dad2561d892b8ddd111f952da694747293a...  
4  ff6da3c2fa482849e6132297244cb0a7de5eaefb4a3773...  
5  9b21c11acaf8985e7ba994f23f134ecee5c5f7777ea

## Step 6: Version Detection Logic

Core logic to detect new pages vs updated pages.

In [33]:
def detect_version_changes(modified_content_df, version_history_df):
    """
    Detect which modified content represents:
    1. NEW pages (first version for a page)
    2. UPDATED pages (subsequent versions)
    3. UNCHANGED pages (checksum matches, skip)
    
    IMPORTANT: Processes rows sorted by last_modified_dt to handle version sequences correctly.
    When multiple rows exist for the same page, earlier rows are treated as previous versions.
    
    Uses CALCULATED_CHECKSUM (from actual files), NOT content_checksum from CSV.
    
    Returns:
        new_pages: List of records that are completely new (first version)
        updated_pages: List of (current_record, previous_version) tuples for updates
        unchanged_pages: List of records with no changes
    """
    new_pages = []
    updated_pages = []
    unchanged_pages = []
    
    # Sort by last_modified_dt to process in chronological order
    modified_content_sorted = modified_content_df.sort_values('last_modified_dt')
    
    # Track pages we've seen in THIS batch (in-memory tracking)
    # Key: (file_name, page_number), Value: latest version record processed
    batch_processed_pages = {}
    
    for _, content in modified_content_sorted.iterrows():
        file_name = content['raw_file_nme']
        page_num = content['raw_file_page_nbr']
        content_id = content['ud_source_file_id']
        checksum = content['calculated_checksum']
        
        # Skip if checksum calculation failed
        if pd.isna(checksum):
            print(f"⚠️ Skipping {file_name} page {page_num}: checksum calculation failed")
            continue
        
        page_key = (file_name, page_num)
        
        # Check if we've already processed this page in the CURRENT batch
        if page_key in batch_processed_pages:
            # This is an UPDATE to a page we saw earlier in this batch
            prev_version = batch_processed_pages[page_key]
            
            if prev_version['content_checksum'] != checksum:
                # CONTENT CHANGED - create new version
                updated_pages.append((content, prev_version))
                # Update batch tracking with this newer version
                batch_processed_pages[page_key] = {
                    'version_history_id': None,  # Will be set after DB insert
                    'version_sequence': int(content['version_nbr']),
                    'content_checksum': checksum,
                    'file_name': file_name,
                    'page_number': page_num
                }
            else:
                # UNCHANGED - skip (checksum matches)
                unchanged_pages.append(content)
            continue
        
        # Check if this file+page combination exists in EXISTING version history (database)
        if len(version_history_df) > 0:
            existing_versions = version_history_df[
                (version_history_df['file_name'] == file_name) &
                (version_history_df['page_number'] == page_num)
            ]
        else:
            existing_versions = pd.DataFrame()
        
        if len(existing_versions) == 0:
            # NEW PAGE - never seen this file+page before (not in DB, not in batch)
            new_pages.append(content)
            # Track this page in batch
            batch_processed_pages[page_key] = {
                'version_history_id': None,
                'version_sequence': int(content['version_nbr']),
                'content_checksum': checksum,
                'file_name': file_name,
                'page_number': page_num
            }
        else:
            # PAGE EXISTS in database - check if content changed
            # latest_version = existing_versions[
            #     existing_versions['is_latest_version'].isin([True, 1])
            # ]
            
            # if len(latest_version) == 0:
            #     # Shouldn't happen, but handle gracefully
            #     latest_version = existing_versions.sort_values('version_sequence', ascending=False).iloc[0]
            # else:
            #     latest_version = latest_version.iloc[0]

            latest_version = existing_versions.sort_values('version_sequence', ascending=False).iloc[0]
            
            if latest_version['content_checksum'] != checksum:
                # CONTENT CHANGED - create new version
                updated_pages.append((content, latest_version))
                # Track this page in batch
                batch_processed_pages[page_key] = {
                    'version_history_id': None,
                    'version_sequence': int(content['version_nbr']),
                    'content_checksum': checksum,
                    'file_name': file_name,
                    'page_number': page_num
                }
            else:
                # UNCHANGED - skip (checksum matches)
                unchanged_pages.append(content)
                # Track this page in batch
                batch_processed_pages[page_key] = {
                    'version_history_id': latest_version['version_history_id'],
                    'version_sequence': latest_version['version_sequence'],
                    'content_checksum': checksum,
                    'file_name': file_name,
                    'page_number': page_num
                }
    
    return new_pages, updated_pages, unchanged_pages

# Run detection
new_pages, updated_pages, unchanged_pages = detect_version_changes(
    modified_content, 
    content_version_history
)

print(f"\n🔍 Detection Results:")
print(f"  📝 New pages (will create v1, v2, ...): {len(new_pages)}")
print(f"  🔄 Updated pages (will create next version): {len(updated_pages)}")
print(f"  ✅ Unchanged pages (skip): {len(unchanged_pages)}")

if len(new_pages) > 0:
    print("\n📝 New Pages:")
    for page in new_pages[:5]:  # Show first 5
        print(f"  - {page['raw_file_nme']}, page {page['raw_file_page_nbr']}, version {page['version_nbr']}")

if len(updated_pages) > 0:
    print("\n🔄 Updated Pages:")
    for content, prev_version in updated_pages[:5]:  # Show first 5
        print(f"  - {content['raw_file_nme']}, page {content['raw_file_page_nbr']} (v{prev_version['version_sequence']} → v{content['version_nbr']})")


🔍 Detection Results:
  📝 New pages (will create v1, v2, ...): 1
  🔄 Updated pages (will create next version): 3
  ✅ Unchanged pages (skip): 2

📝 New Pages:
  - Employee_Leave_Policy.pdf, page 4, version 1

🔄 Updated Pages:
  - Employee_Leave_Policy.pdf, page 3 (v1 → v1)
  - Employee_Leave_Policy.pdf, page 2 (v1 → v2)
  - Employee_Leave_Policy.pdf, page 3 (v1 → v2)


## Step 7: Create Version Records

Build version history records for new and updated pages.

In [17]:
def create_initial_version(content):
    """
    Create version 1 record for a new page.
    Uses CALCULATED checksum from actual markdown file.
    """
    return {
        'content_id': int(content['ud_source_file_id']),
        'previous_version_id': None,
        'version_sequence': int(content['version_nbr']),
        'is_latest_version': 1,  # SQLite uses 1/0 for boolean
        'file_name': content['raw_file_nme'],
        'page_number': int(content['raw_file_page_nbr']),
        'content_checksum': content['calculated_checksum'],  # ← Changed: use calculated checksum
        'change_type': 'initial',
        'change_summary': f"Initial version - {content['raw_file_nme']} page {content['raw_file_page_nbr']}",
        'content_diff': json.dumps({}),
        'version_created_at': content.get('created_dt', content['last_modified_dt']),
        'detected_at': CURRENT_RUN_DATE,
        'created_by': 'system',
        'notes': 'Automatically detected by daily pipeline'
    }

def classify_change_type(content_diff):
    """
    Classify change severity based on diff.
    This is a simple heuristic - can be enhanced with LLM later.
    """
    # Simple classification based on change percentage
    # TODO: Enhance with semantic analysis
    if not content_diff:
        return 'minor'
    
    # For now, return 'major' for any change
    # In production, analyze diff to determine severity
    return 'major'

def create_updated_version(content, previous_version_record):
    """
    Create next version record when content changes.
    Uses CALCULATED checksum from actual markdown file.
    """
    new_version_sequence = int(previous_version_record['version_sequence']) + 1
    
    # Simple diff: just note that it changed
    # TODO: Implement actual content diff by reading markdown files
    content_diff = {
        'type': 'content_change',
        'old_checksum': previous_version_record['content_checksum'],
        'new_checksum': content['calculated_checksum'],  # ← Changed: use calculated checksum
        'changed_at': content['last_modified_dt']
    }
    
    change_type = classify_change_type(content_diff)
    
    return {
        'content_id': int(content['ud_source_file_id']),
        'previous_version_id': int(previous_version_record['version_history_id']),  # Self-referential FK!
        'version_sequence': new_version_sequence,
        'is_latest_version': 1,
        'file_name': content['raw_file_nme'],
        'page_number': int(content['raw_file_page_nbr']),
        'content_checksum': content['calculated_checksum'],  # ← Changed: use calculated checksum
        'change_type': change_type,
        'change_summary': f"Content updated from v{previous_version_record['version_sequence']} to v{new_version_sequence}",
        'content_diff': json.dumps(content_diff),
        'version_created_at': content['last_modified_dt'],
        'detected_at': CURRENT_RUN_DATE,
        'created_by': 'system',
        'notes': f"Automatically detected change from version {previous_version_record['version_sequence']}"
    }

# Create version records
new_version_records = []

# Process new pages
for page in new_pages:
    version_record = create_initial_version(page)
    new_version_records.append(version_record)

# Process updated pages
for content, prev_version in updated_pages:
    version_record = create_updated_version(content, prev_version)
    new_version_records.append(version_record)

print(f"\n📝 Created {len(new_version_records)} version records")

if len(new_version_records) > 0:
    print("\n📋 Sample version records:")
    for record in new_version_records[:3]:
        print(f"  - {record['file_name']} p{record['page_number']}: v{record['version_sequence']}")
        print(f"    Checksum: {record['content_checksum'][:16]}... (calculated from file)")


📝 Created 4 version records

📋 Sample version records:
  - Employee_Leave_Policy.pdf p4: v1
    Checksum: 9b21c11acaf8985e... (calculated from file)
  - Employee_Leave_Policy.pdf p3: v2
    Checksum: 9b21c11acaf8985e... (calculated from file)
  - Employee_Leave_Policy.pdf p2: v2
    Checksum: 02d546c7c11dad25... (calculated from file)


## Step 8: Update Database

Insert new version records and update previous versions.

In [18]:
def update_version_history_db(version_records):
    """
    Update database with new version records.
    
    Steps:
    1. For each new version with a previous version:
       - Set previous version's is_latest_version = 0
    2. Insert new version record
    
    Uses a dedicated connection that is properly closed after operations.
    """
    inserted_count = 0
    updated_count = 0
    
    # Open connection only for this operation, will auto-close
    with sqlite3.connect(VERSION_TRACKING_DB) as conn:
        cursor = conn.cursor()
        
        for record in version_records:
            # 1. Update previous version if exists
            if record['previous_version_id'] is not None:
                cursor.execute("""
                    UPDATE content_version_history
                    SET is_latest_version = 0
                    WHERE version_history_id = ?
                """, (record['previous_version_id'],))
                updated_count += cursor.rowcount
            
            # 2. Insert new version
            cursor.execute("""
                INSERT INTO content_version_history (
                    content_id, previous_version_id, version_sequence,
                    is_latest_version, file_name, page_number,
                    content_checksum, change_type, change_summary,
                    content_diff, version_created_at, detected_at,
                    created_by, notes
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                record['content_id'],
                record['previous_version_id'],
                record['version_sequence'],
                record['is_latest_version'],
                record['file_name'],
                record['page_number'],
                record['content_checksum'],
                record['change_type'],
                record['change_summary'],
                record['content_diff'],
                record['version_created_at'],
                record['detected_at'],
                record['created_by'],
                record['notes']
            ))
            inserted_count += 1
        
        conn.commit()
    
    return inserted_count, updated_count

# Update database
if len(new_version_records) > 0:
    inserted, updated = update_version_history_db(new_version_records)
    print(f"\n✅ Database updated:")
    print(f"  📝 Inserted {inserted} new version records")
    print(f"  🔄 Updated {updated} previous versions (set is_latest_version=0)")
    
    # Verify (using temporary connection)
    with sqlite3.connect(VERSION_TRACKING_DB) as conn_verify:
        total_versions = pd.read_sql("SELECT COUNT(*) as count FROM content_version_history", conn_verify).iloc[0]['count']
    print(f"  📊 Total versions in database: {total_versions}")
else:
    print("\nℹ️ No new versions to insert")

IntegrityError: UNIQUE constraint failed: content_version_history.content_id, content_version_history.version_sequence

## Step 9: FAQ Deprecation (Optional)

Mark FAQs as invalid when their source content changes.

In [ ]:
# Deprecate FAQs (if any pages were updated)
if len(updated_pages) > 0:
    print("\n📍 Deprecating affected FAQs...")
    deprecated_count = 0
    
    with sqlite3.connect(VERSION_TRACKING_DB) as conn_deprecate:
        cursor = conn_deprecate.cursor()
        
        for content, prev_version in updated_pages:
            prev_version_history_id = prev_version['version_history_id']
            
            # Deprecate FAQs linked to previous version
            cursor.execute("""
                UPDATE faq_content_link
                SET is_currently_valid = 0,
                    deprecated_reason = 'content_updated',
                    deprecated_at = ?
                WHERE version_history_id = ?
                  AND is_currently_valid = 1
            """, (CURRENT_RUN_DATE, prev_version_history_id))
            
            deprecated_count += cursor.rowcount
        
        conn_deprecate.commit()
    
    if deprecated_count > 0:
        print(f"  ⚠️ Deprecated {deprecated_count} FAQs due to content changes")
        print("     These FAQs need to be regenerated with updated content.")
    else:
        print("  ℹ️ No FAQs were affected by content changes")
else:
    print("\nℹ️ No pages updated, skipping FAQ deprecation")

## Step 10: Save Run Date & Summary

Save completion timestamp for next run.

In [ ]:
# Save current run date
save_run_date(LAST_RUN_DATE_FILE, CURRENT_RUN_DATE)

# Generate summary report
print("\n" + "="*80)
print("📊 PIPELINE EXECUTION SUMMARY")
print("="*80)

print(f"\n⏰ Execution Time:")
print(f"  Start: {LAST_RUN_DATE}")
print(f"  End: {CURRENT_RUN_DATE}")

print(f"\n📥 Input:")
print(f"  Total content in repo: {len(content_repo)}")
print(f"  Modified since last run: {len(modified_content)}")

print(f"\n🔍 Detection:")
print(f"  New pages (v1): {len(new_pages)}")
print(f"  Updated pages (v2+): {len(updated_pages)}")
print(f"  Unchanged pages: {len(unchanged_pages)}")

print(f"\n💾 Database Updates:")
print(f"  Version records created: {len(new_version_records)}")

with sqlite3.connect(VERSION_TRACKING_DB) as conn_summary:
    total_versions_now = pd.read_sql("SELECT COUNT(*) as count FROM content_version_history", conn_summary).iloc[0]['count']
print(f"  Total versions in DB: {total_versions_now}")

print(f"\n🔗 FAQ Impact:")
try:
    with sqlite3.connect(VERSION_TRACKING_DB) as conn_faq_check:
        deprecated_faqs = pd.read_sql("""
            SELECT COUNT(*) as count FROM faq_content_link 
            WHERE deprecated_at = ?
        """, conn_faq_check, params=(CURRENT_RUN_DATE,)).iloc[0]['count']
    print(f"  FAQs deprecated this run: {deprecated_faqs}")
except:
    print(f"  FAQs deprecated this run: 0")

print(f"\n✅ Pipeline completed successfully")
print("="*80)

## Step 11: Verification Queries

Verify the results of this pipeline run.

In [ ]:
# Query: Latest versions created this run
with sqlite3.connect(VERSION_TRACKING_DB) as conn_verify:
    latest_versions = pd.read_sql("""
        SELECT 
            version_history_id,
            file_name,
            page_number,
            version_sequence,
            change_type,
            change_summary,
            detected_at
        FROM content_version_history
        WHERE detected_at = ?
        ORDER BY version_history_id DESC
    """, conn_verify, params=(CURRENT_RUN_DATE,))

if len(latest_versions) > 0:
    print("\n📋 Versions Created This Run:\n")
    print(latest_versions)
else:
    print("\nℹ️ No versions created this run")

# Query: Version chain example (if any versions were created)
if len(latest_versions) > 0:
    sample_file = latest_versions.iloc[0]['file_name']
    sample_page = latest_versions.iloc[0]['page_number']
    
    with sqlite3.connect(VERSION_TRACKING_DB) as conn_chain:
        version_chain = pd.read_sql("""
            SELECT
                version_history_id,
                content_id,
                previous_version_id,
                version_sequence,
                change_type,
                is_latest_version
            FROM content_version_history
            WHERE file_name = ? AND page_number = ?
            ORDER BY version_sequence
        """, conn_chain, params=(sample_file, sample_page))
    
    print(f"\n🔗 Version Chain Example for '{sample_file}' page {sample_page}:\n")
    print(version_chain)

## Step 12: Cleanup

In [ ]:
print("✅ All database connections have been properly closed using context managers")

print("\n🎉 Daily version tracking pipeline completed successfully!")
print(f"\n💡 Next Run: Process content modified after {CURRENT_RUN_DATE}")